<a href="https://colab.research.google.com/github/emmenets/neuralnetwork/blob/master/chihook190715.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/tensorflow/models.git

In [0]:
!wget https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [5]:
cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
!tar -xf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

In [0]:
mkdir images

In [0]:
mkdir inference_graph

In [0]:
mkdir training

In [10]:
ls

anchor_generators/                   inputs_test.py
box_coders/                          legacy/
builders/                            matchers/
CONTRIBUTING.md                      meta_architectures/
core/                                metrics/
data/                                model_hparams.py
data_decoders/                       model_lib.py
dataset_tools/                       model_lib_test.py
dockerfiles/                         model_main.py
eval_util.py                         models/
eval_util_test.py                    model_tpu_main.py
exporter.py                          object_detection_tutorial.ipynb
exporter_test.py                     predictors/
export_inference_graph.py            protos/
export_tflite_ssd_graph_lib.py       README.md
export_tflite_ssd_graph_lib_test.py  samples/
export_tflite_ssd_graph.py           test_ckpt/
g3doc/                               test_data/
images/                              test_images/
inference/                           tpu_

In [13]:
cd /content/models/research/object_detection

/content/models/research/object_detection


In [0]:
cp  /content/models/research/object_detection/Train-Object-Detection-Classifier/xml_to_csv.py /content/models/research/object_detection

In [38]:
!python3 xml_to_csv.py

Successfully converted xml to csv.
Successfully converted xml to csv.


In [0]:
!cat /content/models/research/object_detection/generate_tfrecord.py

In [0]:
!python generate_tfrecord_01.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

In [0]:
!python generate_tfrecord_01.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record

In [0]:
cp /content/models/research/object_detection/data/pet_label_map.pbtxt /content/models/research/object_detection/training

In [57]:
%%writefile training/label_map.pbtxt

item {
  id: 1
  name: 'chinook'
}

Overwriting training/label_map.pbtxt


In [67]:
%%writefile training/faster_rcnn_inception_v2_pets.config

# Faster R-CNN with Inception v2, configured for Oxford-IIIT Pets Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/content/models/research/object_detection/faster_rcnn_inception_v2_coco_2018_01_28/model.ckpt"
  from_detection_checkpoint: true
  load_all_detection_checkpoint_vars: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}


train_input_reader: {
  tf_record_input_reader {
    input_path: "/content/models/research/object_detection/train.record"
  }
  label_map_path: "/content/models/research/object_detection/training/label_map.pbtxt"
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  num_examples: 70
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/content/models/research/object_detection/test.record"
  }
  label_map_path: "/content/models/research/object_detection/training/label_map.pbtxtt"
  shuffle: false
  num_readers: 1
}

Overwriting training/faster_rcnn_inception_v2_pets.config
